In [ ]:
Assessment

In [5]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate to access Google Docs API
auth.authenticate_user()
docs_service = build('docs', 'v1')

def convert_markdown_to_google_doc(title, markdown_text):
    try:
        # 1. Create a new Google Doc
        doc = docs_service.documents().create(body={'title': title}).execute()
        doc_id = doc.get('documentId')

        lines = markdown_text.split('\n')
        full_text = "\n".join(lines)

        # 2. Initial Text Insertion
        requests = [{'insertText': {'location': {'index': 1}, 'text': full_text}}]

        current_pos = 1
        for line in lines:
            line_len = len(line) + 1

            # Formatting: Headings
            style = None
            if line.startswith('# '): style = 'HEADING_1'
            elif line.startswith('## '): style = 'HEADING_2'
            elif line.startswith('### '): style = 'HEADING_3'

            if style:
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_pos, 'endIndex': current_pos + line_len},
                        'paragraphStyle': {'namedStyleType': style},
                        'fields': 'namedStyleType'
                    }
                })

            # Formatting: Checkboxes
            if "- [ ]" in line:
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': current_pos, 'endIndex': current_pos + line_len},
                        'bulletPreset': 'BULLET_CHECKBOX'
                    }
                })

            # Formatting: Assignee Mentions (@name)
            for match in re.finditer(r'@\w+', line):
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': current_pos + match.start(),
                            'endIndex': current_pos + match.end()
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {'color': {'rgbColor': {'blue': 0.8}}}
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })

            # Formatting: Footer Info
            if "Meeting recorded by" in line or "Duration" in line:
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': current_pos, 'endIndex': current_pos + line_len},
                        'textStyle': {
                            'italic': True,
                            'fontSize': {'magnitude': 10, 'unit': 'PT'}
                        },
                        'fields': 'italic,fontSize'
                    }
                })

            current_pos += line_len

        # Step 3: Execute Batch Update
        docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
        print(f"Success! Document created: https://docs.google.com/document/d/{doc_id}")
        return doc_id

    except HttpError as err:
        print(f"API Error: {err}")

# Define the markdown content
new_markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

convert_markdown_to_google_doc("Product Team Sync - May 15, 2023", new_markdown_content)

Success! Document created: https://docs.google.com/document/d/1MnDg4Xh2hsMs14uywuxW096ihF7TlRBMIhbjaYfafRQ


'1MnDg4Xh2hsMs14uywuxW096ihF7TlRBMIhbjaYfafRQ'